## converting data from xarray to pandas format

In [ ]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt


### Saildrone data in xarray form

In [ ]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds = xr.open_dataset(url)
ds

In [ ]:
# swap obs for time
ds=ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})
ds

In [ ]:
#resample to make time = 1 hr increments
hr_resample=ds.resample(time='1h', skipna=True, label='right').mean()
ds

In [ ]:
##do the math stuff in xarray
#then save as a dataframe, then to csv

### Decode and store data as Pandas

In [ ]:
pd_ds = ds.to_dataframe()
pd_ds

In [ ]:
pd_ds[['time', 'PITCH']]

In [ ]:
#write dataframe to file
pd_ds.to_csv("test1.csv")  
#read it back
pd.read_csv("test1.csv").head()

Convert new csv file to a text file

In [ ]:
# A simple program to create a formatted text file from a *.csv file.

csv_file = input('test1.csv')
txt_file = input('test1.txt')

try:
    my_input_file = open(csv_file, "r")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_input_file.closed:
    text_list = [];
    for line in my_input_file.readlines():
        line = line.split(",", 2)
        text_list.append(" ".join(line))
    my_input_file.close()

try:
    my_output_file = open(txt_file, "w")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_output_file.closed:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')
    my_output_file.close()